# Notebook for exploring different models and prompts

## Setup

In [4]:
# SQLite compatibility fix for Streamlit Cloud
import sys
sys.path.append('..') 

try:
    __import__('pysqlite3')
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
except ImportError:
    pass

from src.rag_system import RAGSystem
from src.document_processor import DocumentProcessor
from src.vector_store import VectorStore

In [5]:
rag_system = RAGSystem()

rag_system.get_stats()

HUGGINGFACE_API_TOKEN not found in environment variables


ValueError: HuggingFace API token is required